In [1]:
!python -m pip install "xarray[parallel]"

In [2]:
!pip list

Package                       Version
----------------------------- -------------------
aiobotocore                   2.4.2
aiohttp                       3.8.3
aioitertools                  0.11.0
aiosignal                     1.3.1
anyio                         2.1.0
argon2-cffi                   20.1.0
async-generator               1.10
async-timeout                 4.0.2
attrs                         20.3.0
Babel                         2.9.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.1
bleach                        3.2.1
bokeh                         2.4.3
botocore                      1.27.59
branca                        0.4.2
brotlipy                      0.7.0
certifi                       2020.12.5
cffi                          1.14.4
chardet                       4.0.0
charset-normalizer            2.1.1
click                         8.1.3
cloudpickle                   2.2.1
conda                         4.9.2
conda-package-handling        1.7.2
cry

In [3]:
import xarray as xr

In [4]:
!ls

'Put NCAR in Ceph.ipynb'   con_stats.mod   read_ncar.f
 a.out			   go_compile	   sciserver-experimental
 ariel			   jhd.100.nc	   turbulence_output
 ceph_s3_creds.txt	   jobs		   turbulence_pickled_zarr_test
 con_data.mod		   pars.mod	   vars.mod


In [5]:
ds = xr.open_dataset("./jhd.100.nc", chunks="auto")

In [6]:
# Ryan's Ceph code
data_xr = ds

In [7]:
import os
 
import numpy as np
import s3fs
import zarr
import xarray as xr
import dask
 
 
# sets s3 login env variables
with open("ceph_s3_creds.txt", "r") as f:
    for l in f.readlines():
        k, v = l.strip().split("=")
        os.environ[k] = v
 
 
# get your data from where
# data = np.zeros([512, 512, 512, 3])

s3fs.S3FileSystem.connect_timeout = 0.1
 
 
# sample 1
# SAVING WITH XARRAY vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
s3 = s3fs.S3FileSystem(
    anon=False,
    client_kwargs=dict(
        endpoint_url='https://rgw.idies.jhu.edu',
    )
)
 
# this is a made up key that will act like a path to the zarr files
path_key = "sciserver-experimental/ariel/tests"
 
store = zarr.storage.FSStore(
    path_key,
    mode="w",
    fs=s3
)
 
# data_xr = xr.DataArray(
#     data,
#     dims=("z", "y", "x", "c"),
#     name=variable,
# ).to_dataset()

# data_xr = data_xr.to_dataset()

# tqdm to check progress
# data_xr.to_zarr(
#      store=store,
#      mode="w"#,
# #      encoding={
# #         variable: dict(chunks=(64, 64, 64, 3), compressor=None)
# #      }
# )

# https://docs.dask.org/en/stable/diagnostics-local.html
from dask.diagnostics import ProgressBar

# https://github.com/pangeo-data/pangeo/issues/150
# with dask.config.set(get=dask.threaded.get):
with ProgressBar():
    data_xr.to_zarr(store=store, mode="w")# overwrite if exists

[########################################] | 100% Completed |  1hr 54min 57.7s
